# Scenariusze testowe dla porównania wydajności baz danych

### 1. Operacja CREATE

- Dodanie nowego nauczyciela
- Utworzenie nowej klasy
- Dodanie nowego przedmiotu
- Zarejestrowanie nowego ucznia
- Przypisanie ucznia do klasy (**Dodano: Zapisanie ucznia do klasy (enrolment)**)
- Utworzenie harmonogramu zajęć
- Wystawienie oceny

### 2. Operacja READ

Pobranie kompleksowego raportu zawierającego:
- Dane osobowe ucznia
- Informacje o klasie (**Dodano: Informacje o zapisach do klas**)
- Dane nauczyciela prowadzącego
- Listę ocen z opisami przedmiotów
- Szczegółowy harmonogram zajęć

### 3. Operacja UPDATE

- Aktualizacja danych ucznia
- Zmiana przypisania do klasy (**Dodano: Aktualizacja zapisu do klasy**)
- Modyfikacja nazwy klasy
- Aktualizacja danych nauczyciela
- Zmiana oceny
- Aktualizacja opisu przedmiotu
- Modyfikacja harmonogramu zajęć

### 4. Operacja DELETE

- Usunięcie ocen ucznia
- Wypisanie ucznia z klasy (**Dodano: Usunięcie zapisu do klasy**)
- Usunięcie harmonogramu zajęć
- Usunięcie klasy
- Opcjonalne usunięcie przedmiotów
- Opcjonalne usunięcie nauczyciela
- Usunięcie rekordu ucznia

## Ilość rekordów do testów

Testy będą przeprowadzane dla następujących ilości rekordów:

1. 10,000 rekordów
2. 100,000 rekordów
3. 1,000,000 rekordów
4. 10,000,000 rekordów

## Metryki wydajnościowe

Dla każdego scenariusza i ilości rekordów będziemy mierzyć:

1. Czas wykonania całego scenariusza
2. Średni czas pojedynczych operacji
3. Liczbę operacji na sekundę (throughput)
4. Zużycie zasobów systemowych (CPU, RAM, I/O dysku)

# Narzędzia i technologie testowe

### Wbudowane instrumenty bazodanowe

Każdy system oferuje specjalizowane narzędzia diagnostyczne:

| System | Narzędzie | Funkcjonalności |
| :-- | :-- | :-- |
| PostgreSQL | pgBench | Testy TPC-B, własne skrypty SQL |
| MariaDB | sysbench | Testy OLTP, skalowanie pionowe |
| MongoDB | mongoperf | Operacje na dokumentach JSON |
| Cassandra | cassandra-stress | Testy dystrybucji danych |
| Redis | redis-benchmark | Pomiar opóźnień operacji klucz-wartość |

Wykorzystanie natywnych narzędzi pozwala na precyzyjne badanie specyficznych mechanizmów storage engine.

### Automatyzacja w Pythonie

Kluczowe biblioteki wspierające testy:

- **SQLAlchemy** dla baz relacyjnych
- **PyMongo** dla MongoDB
- **Cassandra-driver** dla Cassandra
- **redis-py** dla Redis

In [ ]:
# Import required libraries
import psycopg2
import psycopg2.errors
from pymongo import MongoClient
from cassandra.cluster import Cluster
import redis
import mysql.connector
import yaml
import pandas as pd
import os
import time
import sys
from pathlib import Path

# Load database configuration
print("Setting up database connections...")
with open('docker-compose.yml', 'r') as file:
    docker_config = yaml.safe_load(file)

# PostgreSQL connection
postgres_config = docker_config['services']['postgresql']
postgres_client = psycopg2.connect(
    host='localhost',
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTGRES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

# MariaDB connection
mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0],
    allow_local_infile=True
)

# MongoDB connection
mongo_config = docker_config['services']['mongodb']
mongo_client = MongoClient(
    host='localhost',
    port=int(mongo_config['ports'][0].split(':')[0])
)

# Cassandra connection
cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'], port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

# Redis connection
redis_config = docker_config['services']['redis']
redis_client = redis.Redis(
    host='localhost',
    port=int(redis_config['ports'][0].split(':')[0])
)

# Test connections
try:
    postgres_client.cursor().execute("SELECT 1")
    print("INFO: PostgreSQL connection successful")
    
    mariadb_client.cursor(buffered=True).execute("SELECT 1")
    print("INFO: MariaDB connection successful")
    
    cassandra_session.execute("SELECT release_version FROM system.local")
    print("INFO: Cassandra connection successful")
    
    mongo_client.admin.command('ping')
    print("INFO: MongoDB connection successful")
    
    redis_client.ping()
    print("INFO: Redis connection successful")
except Exception as e:
    print(f"ERROR: Connection test failed: {e}")

In [ ]:
CELL_END = ''

In [ ]:
# Data generation functions
sys.path.append(str(Path.cwd()))
from generator import generate_school_data

def generate_files(output_dir='./data', scale=1000, batch_size=10000, **kwargs):
    """
    Generate synthetic school data files for benchmarking.
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    print(f"INFO: Generating data with scale {scale} and batch size {batch_size}...")

    result = generate_school_data(
        output_dir=output_dir,
        scale=scale,
        batch_size=batch_size,
        **kwargs
    )

    print(f"INFO: Generated {len(result['students'])} students, {len(result['teachers'])} teachers, " + 
        f"{len(result['classes'])} classes, {len(result['subjects'])} subjects")
    print("="*50)
    return result

# Generate test data sets
scale_100_dir = './data/scale_100'

generate_files(output_dir=scale_100_dir, scale=100, batch_size=5000)
CELL_END


# PostgreSQL Operations

In [ ]:
# PostgreSQL Methods

def initialize_postgres_schema(conn, schema_sql):
    """
    Initializes the PostgreSQL database schema using the provided SQL script.
    """
    if not schema_sql:
        print("ERROR: Schema SQL content is empty.")
        return

    try:
        with conn.cursor() as cur:
            cur.execute(schema_sql)
        conn.commit()
        print("INFO: PostgreSQL schema initialized.")
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error initializing PostgreSQL schema: {e}")

def verify_postgres_tables(conn, expected_tables):
    """
    Verifies if the expected tables exist in PostgreSQL.
    """
    try:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public' AND table_name = ANY(%s);
            """, (expected_tables,))
            existing_tables = {row[0] for row in cur.fetchall()}

        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All PostgreSQL tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing PostgreSQL tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: Error verifying PostgreSQL tables: {e}")
        return False

def insert_postgres_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a PostgreSQL table using INSERT.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            with open(csv_file, 'r') as f:
                next(f)  # Skip header
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                for line in f:
                    values = line.strip().split(',')
                    insert_sql = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(values))})"
                    try:
                        cur.execute(insert_sql, values)
                    except psycopg2.errors.UniqueViolation:
                        # Ignore duplicate key errors
                        conn.rollback()
                        continue
                    except Exception as e:
                        print(f"ERROR: Error inserting into {table_name}: {e}")
                        conn.rollback()
                        break
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_postgres_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a PostgreSQL table using COPY.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            copy_sql = f"COPY {table_name} FROM STDIN WITH (FORMAT CSV, HEADER)"
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                cur.copy_expert(sql=copy_sql, file=f)
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_postgres_enrollments_from_csv(conn, csv_file):
    # This function is a specialized version for the enrollments table, because it has a composite primary key.
    # Inserts data from a CSV file into the enrollments table using COPY.
    # Does not fail on duplicate key errors.
    # Uses a temporary table to handle duplicates.

    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time

    try:
        with conn.cursor() as cur:
            # Create a temporary table for the COPY operation
            temp_table_name = "temp_enrollments"
            cur.execute(f"""
                CREATE TEMP TABLE {temp_table_name} (
                    student_id INT,
                    class_id INT,
                    enrolled_at TIMESTAMP
                ) ON COMMIT DROP;
            """)
            copy_sql = f"COPY {temp_table_name} FROM STDIN WITH (FORMAT CSV, HEADER)"
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time()
                cur.copy_expert(sql=copy_sql, file=f)

            # Insert into the main table, ignoring duplicates
            insert_sql = f"""
                INSERT INTO enrollments (student_id, class_id, enrolled_at)
                SELECT student_id, class_id, enrolled_at FROM {temp_table_name}
                ON CONFLICT (student_id, class_id) DO NOTHING;
            """
            cur.execute(insert_sql)
            conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into enrollments: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def load_postgres_data(conn, data_dir):
    """
    Loads data from CSV files into PostgreSQL tables.
    """
    data_path = Path(data_dir)
    table_csv_map = {
        'teachers': 'teachers.csv',
        'subjects': 'subjects.csv',
        'classes': 'classes.csv',
        'students': 'students.csv',
        'grades': 'grades.csv',
        'schedules': 'schedules.csv',
        # 'enrollments': 'enrollments.csv' # Handled separately
    }
    for table_name, csv_file in table_csv_map.items():
        op_time, f_op_time, end_time = copy_postgres_table_from_csv(conn, table_name, data_path / csv_file)
        print(f"INFO: Inserted {table_name} in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

    # Handle enrollments separately due to composite primary key
    op_time, f_op_time, end_time = copy_postgres_enrollments_from_csv(conn, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

def verify_postgres_counts(conn, tables):
    """
    Counts rows in PostgreSQL tables.
    """
    counts = {}
    max_len = max(len(t) for t in tables) if tables else 0
    print(f"INFO: Counting rows in PostgreSQL tables")
    try:
        with conn.cursor() as cur:
            for table_name in tables:
                try:
                    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
                    count = cur.fetchone()[0]
                    counts[table_name] = count
                except Exception as count_error:
                    print(f"ERROR: {count_error}")
                    counts[table_name] = 'Error'

        print("--- PostgreSQL Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("-----------------------------------")
        return counts

    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [ ]:
# PostgreSQL Operations Execution

# Schema initialization
with open('schemas/postgres_schema.sql', 'r') as f:
    sql_schema = f.read()

initialize_postgres_schema(postgres_client, sql_schema)

# Table verification 
required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
verify_postgres_tables(postgres_client, required_tables)

# Data loading
load_postgres_data(postgres_client, scale_100_dir)

# Count verification
verify_postgres_counts(postgres_client, required_tables)
CELL_END

# MariaDB Operations

In [ ]:
# MariaDB Methods

def initialize_mariadb_schema(conn, schema_sql):
    """
    Initializes the MariaDB database schema using the provided SQL script.
    """
    if not schema_sql:
        print("ERROR: Schema SQL content is empty.")
        return
    try:
        with conn.cursor() as cur:
            for statement in schema_sql.split(';'):
                stmt = statement.strip()
                if stmt:
                    cur.execute(stmt)
        conn.commit()
        print("INFO: MariaDB schema initialized.")
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error initializing MariaDB schema: {e}")

def verify_mariadb_tables(conn, expected_tables):
    """
    Verifies if the expected tables exist in MariaDB.
    """
    try:
        with conn.cursor() as cur:
            format_strings = ','.join(['%s'] * len(expected_tables))
            cur.execute(f"""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = DATABASE() AND table_name IN ({format_strings});
            """, tuple(expected_tables))
            existing_tables = {row[0] for row in cur.fetchall()}

        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All MariaDB tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing MariaDB tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: Error verifying MariaDB tables: {e}")
        return False

def insert_mariadb_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    """Inserts data from a CSV file into a MariaDB table by reading the header for columns."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    try:
        with conn.cursor() as cur:
            with open(csv_file, 'r') as f:
                # read header for column names
                header = next(f).strip().split(',')
                cols = header
                placeholders = ','.join(['%s'] * len(cols))
                insert_sql = f"INSERT INTO {table_name} ({','.join(cols)}) VALUES ({placeholders})"
                file_opened_start_time = time.time()
                for line in f:
                    values = line.strip().split(',')
                    # ensure values length matches columns
                    if len(values) != len(cols):
                        if len(values) > len(cols):
                            values = values[:len(cols)]
                        else:
                            print(f"WARNING: Skipping {table_name} row with {len(values)} values (expected {len(cols)}, values: {values})")
                            continue
                    try:
                        cur.execute(insert_sql, values)
                    except mysql.connector.errors.IntegrityError:
                        conn.rollback()
                        continue
                    except Exception as e:
                        print(f"ERROR: Error inserting into {table_name}: {e}")
                        conn.rollback()
                        break
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)


def copy_mariadb_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a MariaDB table using COPY.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            copy_sql = f"""
            LOAD DATA LOCAL INFILE '{csv_file}'
            INTO TABLE {table_name}
            FIELDS TERMINATED BY ','
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            IGNORE 1 LINES;
            """
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                cur.execute(copy_sql)
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_mariadb_enrollments_from_csv(conn, csv_file):
    # This function is a specialized version for the enrollments table, because it has a composite primary key.
    # Inserts data from a CSV file into the enrollments table using COPY.
    # Does not fail on duplicate key errors.
    # Uses a temporary table to handle duplicates.

    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time

    try:
        with conn.cursor() as cur:
            # Handle enrollments with INSERT IGNORE to skip duplicates
            print(f"INFO: Loading enrollments with duplicate handling...")
            with open(csv_file, 'r') as f:
                next(f)  # skip header
                for line in f:
                    student_id, class_id, enrolled_at = line.strip().split(',')
                    cur.execute(
                        """
                        INSERT IGNORE INTO enrollments (student_id, class_id, enrolled_at)
                        VALUES (%s, %s, %s)
                        """,
                        (student_id, class_id, enrolled_at)
                    )
            conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into enrollments: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def load_mariadb_data(conn, data_dir):
    table_csv_map = {
    'teachers': 'teachers.csv',
    'subjects': 'subjects.csv',
    'classes': 'classes.csv',
    'students': 'students.csv',
    'grades': 'grades.csv',
    'schedules': 'schedules.csv',
    # 'enrollments': 'enrollments.csv' Handled separately
    }
    data_path = Path(data_dir)
    for table_name, csv_file in table_csv_map.items():
        op_time, f_op_time, end_time = insert_mariadb_table_from_csv(conn, table_name, data_path / csv_file)
        print(f"INFO: Inserted {table_name} in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

    # Handle enrollments separately due to composite primary key
    op_time, f_op_time, end_time = copy_mariadb_enrollments_from_csv(conn, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

def verify_mariadb_counts(conn, tables):
    """
    Counts rows in MariaDB tables.
    """
    counts = {}
    max_len = max(len(t) for t in tables) if tables else 0
    print(f"INFO: Counting rows in MariaDB tables")
    
    try:
        with conn.cursor() as cur:
            for table_name in tables:
                try:
                    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
                    count = cur.fetchone()[0]
                    counts[table_name] = count
                except Exception as count_error:
                    print(f"ERROR: {count_error}")
                    counts[table_name] = 'Error'

        print("--- MariaDB Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("---------------------------------")
        return counts

    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [ ]:
# # MariaDB Operations Execution

# # Schema initialization
# with open('schemas/mariadb_schema.sql', 'r') as f:
#     mariadb_schema = f.read()

# initialize_mariadb_schema(mariadb_client, mariadb_schema)

# # Table verification
# required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
# verify_mariadb_tables(mariadb_client, required_tables)

# # Data loading
# load_mariadb_data(mariadb_client, scale_100_dir)

# # Count verification
# verify_mariadb_counts(mariadb_client, required_tables)
# CELL_END

# MongoDB Operations

In [ ]:
# MongoDB Methods
def initialize_mongo_schema(client, db_name='benchmark'):
    """
    Initializes the MongoDB schema by creating necessary collections.
    """
    try:
        db = client[db_name]
        
        # List of collections to create based on no_sql_design.txt
        collections = ['students', 'teachers', 'classes', 'subjects']
        
        # Drop existing collections if they exist
        for collection in collections:
            if collection in db.list_collection_names():
                db[collection].drop()
                print(f"INFO: Dropped MongoDB collection: {collection}")
        
        # Create collections with indexes
        for collection in collections:
            db.create_collection(collection)
            print(f"INFO: Created MongoDB collection: {collection}")
            
            # Create indexes for performance
            if collection == 'students':
                db[collection].create_index([("last_name", 1), ("first_name", 1)])
            elif collection == 'classes':
                db[collection].create_index([("name", 1)])
                
        print("INFO: MongoDB schema initialized.")
    except Exception as e:
        print(f"ERROR: {e}")

def verify_mongo_collections(client, db_name='benchmark', expected_collections=None):
    """
    Verifies if the expected collections exist in MongoDB.
    """
    if expected_collections is None:
        expected_collections = ['students', 'teachers', 'classes', 'subjects']
    
    try:
        db = client[db_name]
        existing_collections = db.list_collection_names()
        
        missing_collections = set(expected_collections) - set(existing_collections)
        if not missing_collections:
            print(f"INFO: All MongoDB collections exist: {', '.join(expected_collections)}")
            return True
        else:
            print(f"WARNING: Missing MongoDB collections: {', '.join(missing_collections)}")
            return False
    except Exception as e:
        print(f"ERROR: {e}")
        return False

def insert_mongo_data_from_csv(client, collection_name, csv_file) -> tuple[float, float, float]:
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        db = client['benchmark']
        collection = db[collection_name]
        
        with open(csv_file, 'r') as f:
            reader = pd.read_csv(f)
            # rename id to _id for MongoDB
            if 'id' in reader.columns:
                reader.rename(columns={'id': '_id'}, inplace=True)

            file_opened_start_time = time.time() # Initialize start_time just before starting to insert
            for _, row in reader.iterrows():
                doc = row.to_dict()
                collection.insert_one(doc)
    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def insert_mongo_students_from_csv(client, data_path) -> tuple[float, float]:
    # load all grades and enrollments into students from csv files
    # create a student object with embedded enrollments and grades
    students_file = data_path / 'students.csv'
    enrollments_file = data_path / 'enrollments.csv'
    grades_file = data_path / 'grades.csv'
    operation_start_time = time.time() # Initialize start_time

    try:
        db = client['benchmark']
        collection = db['students']
        
        with open(students_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_doc = {
                    "_id": row['id'],
                    "first_name": row['first_name'],
                    "last_name": row['last_name'],
                    "birth_date": row['birth_date'],
                    "enrollments": [],
                    "grades": []
                }
                collection.insert_one(student_doc)

        with open(enrollments_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_id = row['student_id']
                enrollment_doc = {
                    "class_id": row['class_id'],
                    "enrolled_at": row['enrolled_at']
                }
                collection.update_one(
                    {"_id": student_id},
                    {"$push": {"enrollments": enrollment_doc}}
                )

        with open(grades_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_id = row['student_id']
                grade_doc = {
                    "subject_id": row['subject_id'],
                    "grade": row['grade'],
                    "created_at": row['created_at']
                }
                collection.update_one(
                    {"_id": student_id},
                    {"$push": {"grades": grade_doc}}
                )

    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, end_time)


def insert_mongo_classes_from_csv(client, data_path) -> tuple[float, float]:
    # load all teachers and schedules into classes from csv files
    # create a class object with embedded teachers and schedules
    classes_file = data_path / 'classes.csv'
    schedules_file = data_path / 'schedules.csv'
    operation_start_time = time.time() # Initialize start_time

    try:
        db = client['benchmark']
        collection = db['classes']

        with open(classes_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                class_doc = {
                    "_id": row['id'],
                    "name": row['name'],
                    "teacher_id": row['teacher_id'],
                    "schedule": []
                }
                collection.insert_one(class_doc)

        with open(schedules_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                class_id = row['class_id']
                schedule_doc = {
                    "subject_id": row['subject_id'],
                    "day_of_week": row['day_of_week'],
                    "time_start": row['time_start'],
                    "time_end": row['time_end']
                }
                collection.update_one(
                    {"_id": class_id},
                    {"$push": {"schedule": schedule_doc}}
                )

    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, end_time)

def load_mongo_data(client, data_dir):
    data_path = Path(data_dir)
    insert_mongo_data_from_csv(client, 'teachers', data_path / 'teachers.csv')
    insert_mongo_data_from_csv(client, 'subjects', data_path / 'subjects.csv')
    insert_mongo_students_from_csv(client, data_path)
    insert_mongo_classes_from_csv(client, data_path)

def verify_mongo_counts(client, db_name='benchmark'):
    """
    Counts documents in MongoDB collections.
    """
    collections = ['students', 'teachers', 'classes', 'subjects']
    max_len = max(len(c) for c in collections)
    
    try:
        db = client[db_name]
        counts = {}
        
        for collection in collections:
            try:
                count = db[collection].count_documents({})
                counts[collection] = count
            except Exception as e:
                print(f"ERROR: {e}")
                counts[collection] = 'Error'
                
        print("--- MongoDB Collection Document Counts ---")
        for collection, count in counts.items():
            print(f"{collection:<{max_len}} : {count}")
        print("-----------------------------------------")

        # Additional checks for embedded documents
        try:
            students_with_enrollments = db.students.count_documents({"enrollments": {"$exists": True, "$ne": []}})
            students_with_grades = db.students.count_documents({"grades": {"$exists": True, "$ne": []}})
            classes_with_schedules = db.classes.count_documents({"schedule": {"$exists": True, "$ne": []}})
            
            print("\n--- MongoDB Embedded Document Counts ---")
            print(f"Students with enrollments : {students_with_enrollments}")
            print(f"Students with grades      : {students_with_grades}")
            print(f"Classes with schedules    : {classes_with_schedules}")
            print("-----------------------------------------")
        except Exception as e:
            print(f"ERROR: {e}")
        
        return counts
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [ ]:
# MongoDB Operations Execution

# Schema initialization
initialize_mongo_schema(mongo_client)

# Collection verification
verify_mongo_collections(mongo_client)

# Data loading
load_mongo_data(mongo_client, scale_100_dir)

# Document count verification
verify_mongo_counts(mongo_client)
CELL_END

In [ ]:
# Cassandra data loading functions with minimal memory usage

def initialize_cassandra_schema(session, keyspace='benchmark'):
    """Initializes the Cassandra schema by creating necessary keyspace and tables."""
    try:
        # Create keyspace if not exists
        session.execute(f"""
            CREATE KEYSPACE IF NOT EXISTS {keyspace} 
            WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }};
        """)
        
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Drop existing tables if they exist
        tables = ['teachers', 'subjects', 'classes', 'students', 
                    'enrollments', 'grades', 'schedules', 
                    'student_enrollments', 'student_grades']
        
        for table in tables:
            session.execute(f"DROP TABLE IF EXISTS {table};")
            print(f"INFO: Dropped Cassandra table: {table}")
        
        # Create tables with appropriate data types
        session.execute("""
            CREATE TABLE teachers (
                id INT PRIMARY KEY,
                first_name TEXT,
                last_name TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE subjects (
                id INT PRIMARY KEY,
                name TEXT,
                description TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE classes (
                id INT PRIMARY KEY,
                name TEXT,
                teacher_id INT
            );
        """)
        
        session.execute("""
            CREATE TABLE students (
                id INT PRIMARY KEY,
                first_name TEXT,
                last_name TEXT,
                birth_date TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE enrollments (
                student_id INT,
                class_id INT,
                enrolled_at TIMESTAMP,
                PRIMARY KEY (student_id, class_id)
            );
        """)
        
        session.execute("""
            CREATE TABLE grades (
                id INT PRIMARY KEY,
                student_id INT,
                subject_id INT,
                grade FLOAT,
                created_at TIMESTAMP
            );
        """)
        
        session.execute("""
            CREATE TABLE schedules (
                id INT PRIMARY KEY,
                class_id INT,
                subject_id INT,
                day_of_week INT,
                time_start TEXT,
                time_end TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE student_enrollments (
                student_id INT,
                class_id INT,
                class_name TEXT,
                teacher_id INT,
                enrolled_at TIMESTAMP,
                PRIMARY KEY (student_id, class_id)
            );
        """)
        
        session.execute("""
            CREATE TABLE student_grades (
                student_id INT,
                subject_id INT,
                subject_name TEXT,
                grade FLOAT,
                created_at TIMESTAMP,
                PRIMARY KEY (student_id, subject_id, created_at)
            ) WITH CLUSTERING ORDER BY (subject_id ASC, created_at DESC);
        """)
        
        print("INFO: Cassandra schema initialized.")
    except Exception as e:
        print(f"ERROR: {e}")

def verify_cassandra_tables(session, keyspace='benchmark', expected_tables=None):
    """Verifies if the expected tables exist in Cassandra."""
    if expected_tables is None:
        expected_tables = ['teachers', 'subjects', 'classes', 'students', 
                            'enrollments', 'grades', 'schedules']
    
    try:
        # Get existing tables
        query = f"""
            SELECT table_name FROM system_schema.tables 
            WHERE keyspace_name = '{keyspace}';
        """
        rows = session.execute(query)
        existing_tables = {row.table_name for row in rows}
        
        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All Cassandra tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing Cassandra tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: {e}")
        return False

def insert_cassandra_teachers(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert teacher data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO teachers (id, first_name, last_name) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # first_name
                        values[2]             # last_name
                    ])
        
        print("INFO: Inserted teachers successfully")
    except Exception as e:
        print(f"ERROR: Failed to load teachers: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_subjects(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert subject data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO subjects (id, name, description) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # name
                        values[2]             # description
                    ])
        
        print("INFO: Inserted subjects successfully")
    except Exception as e:
        print(f"ERROR: Failed to load subjects: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_classes(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert class data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO classes (id, name, teacher_id) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # name
                        int(values[2])        # teacher_id
                    ])
        
        print("INFO: Inserted classes successfully")
    except Exception as e:
        print(f"ERROR: Failed to load classes: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_students(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert student data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO students (id, first_name, last_name, birth_date) VALUES (?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 4:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # first_name
                        values[2],            # last_name
                        values[3]             # birth_date
                    ])
        
        print("INFO: Inserted students successfully")
    except Exception as e:
        print(f"ERROR: Failed to load students: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_enrollments(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert enrollment data from CSV, line by line, with timestamp handling."""
    from datetime import datetime
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO enrollments (student_id, class_id, enrolled_at) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    # Convert timestamp string to datetime object
                    enrolled_at = datetime.fromisoformat(values[2].replace('Z', '+00:00'))
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),       # student_id
                        int(values[1]),       # class_id
                        enrolled_at           # enrolled_at as datetime
                    ])
        
        print("INFO: Inserted enrollments successfully")
    except Exception as e:
        print(f"ERROR: Failed to load enrollments: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_grades(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert grades data from CSV, line by line, with UUID and timestamp handling."""
    from datetime import datetime
    from uuid import uuid4
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO grades (id, student_id, subject_id, grade, created_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 4:  # Ensure we have enough columns
                    # Convert timestamp string to datetime object
                    created_at = datetime.fromisoformat(values[4].replace('Z', '+00:00'))
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),              # id (generated UUID)
                        int(values[1]),       # student_id
                        int(values[2]),       # subject_id
                        float(values[3]),     # grade
                        created_at            # created_at as datetime
                    ])
        
        print("INFO: Inserted grades successfully")
    except Exception as e:
        print(f"ERROR: Failed to load grades: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_schedules(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert schedule data from CSV, line by line, with UUID and day mapping."""
    from uuid import uuid4
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Day name to integer mapping
        day_map = {
            'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 
            'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7
        }
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO schedules (id, class_id, subject_id, day_of_week, time_start, time_end) VALUES (?, ?, ?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 6:  # Ensure we have enough columns
                    # Convert day name to integer
                    day_num = day_map.get(values[3], 0)
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),              # id (generated UUID)
                        int(values[1]),       # class_id
                        int(values[2]),       # subject_id
                        day_num,              # day_of_week as int
                        values[4],            # time_start
                        values[5]             # time_end
                    ])
        
        print("INFO: Inserted schedules successfully")
    except Exception as e:
        print(f"ERROR: Failed to load schedules: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def populate_cassandra_denormalized_tables(session, data_dir, keyspace='benchmark'):
    """
    Populate denormalized tables for efficient queries.
    """
    from datetime import datetime
    import pandas as pd
    
    data_path = Path(data_dir)
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # --- Populate student_enrollments table ---
        print("INFO: Populating student_enrollments table...")
        enrollments_df = pd.read_csv(data_path / 'enrollments.csv')
        classes_df = pd.read_csv(data_path / 'classes.csv')
        
        # Prepare statement
        stmt = session.prepare(
            "INSERT INTO student_enrollments (student_id, class_id, class_name, teacher_id, enrolled_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Join and process
        merged = pd.merge(enrollments_df, classes_df, left_on='class_id', right_on='id')
        
        # Process in batches to avoid memory issues
        batch_size = 100
        for i in range(0, len(merged), batch_size):
            batch = merged.iloc[i:i+batch_size]
            for _, row in batch.iterrows():
                try:
                    enrolled_at = datetime.fromisoformat(row['enrolled_at'].replace('Z', '+00:00'))
                    
                    session.execute(stmt, [
                        int(row['student_id']),
                        int(row['class_id']),
                        str(row['name']),
                        int(row['teacher_id']),
                        enrolled_at
                    ])
                except Exception as e:
                    print(f"WARNING: Error inserting row into student_enrollments: {e}")
        
        print("INFO: Populated student_enrollments denormalized table")
        
        # --- Populate student_grades table ---
        print("INFO: Populating student_grades table...")
        grades_df = pd.read_csv(data_path / 'grades.csv')
        subjects_df = pd.read_csv(data_path / 'subjects.csv')
        
        # Prepare statement
        stmt = session.prepare(
            "INSERT INTO student_grades (student_id, subject_id, subject_name, grade, created_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Join and process - use explicit column references to avoid ambiguity
        merged = pd.merge(grades_df, subjects_df, left_on='subject_id', right_on='id', suffixes=('_grade', '_subject'))
        
        # Process in batches to avoid memory issues
        for i in range(0, len(merged), batch_size):
            batch = merged.iloc[i:i+batch_size]
            for _, row in batch.iterrows():
                try:
                    # Explicitly handle data types to avoid conversion errors and use the renamed column
                    student_id = int(row['student_id'])
                    subject_id = int(row['subject_id'])
                    subject_name = str(row['name'])
                    grade = float(row['grade'])
                    created_at = datetime.fromisoformat(str(row['created_at_grade']).replace('Z', '+00:00'))
                    
                    session.execute(stmt, [
                        student_id,
                        subject_id,
                        subject_name,
                        grade,
                        created_at
                    ])
                except Exception as e:
                    # Add debug info to see column names
                    print(f"WARNING: Error inserting row into student_grades: {e}")
                    print(f"DEBUG: Available columns: {list(row.index)}")
        
        print("INFO: Populated student_grades denormalized table")
        
    except Exception as e:
        print(f"ERROR: Failed to populate denormalized tables: {e}")
        import traceback
        traceback.print_exc()

def load_cassandra_data(session, data_dir, keyspace='benchmark'):
    """Load all data into Cassandra tables."""
    data_path = Path(data_dir)
    
    # Use the keyspace
    session.execute(f"USE {keyspace};")
    
    # Insert basic entities
    op_time, f_op_time, end_time = insert_cassandra_teachers(session, data_path / 'teachers.csv')
    print(f"INFO: Inserted teachers in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_subjects(session, data_path / 'subjects.csv')
    print(f"INFO: Inserted subjects in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_classes(session, data_path / 'classes.csv')
    print(f"INFO: Inserted classes in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_students(session, data_path / 'students.csv')
    print(f"INFO: Inserted students in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Insert relationships and complex data
    op_time, f_op_time, end_time = insert_cassandra_enrollments(session, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_grades(session, data_path / 'grades.csv')
    print(f"INFO: Inserted grades in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_schedules(session, data_path / 'schedules.csv')
    print(f"INFO: Inserted schedules in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Populate denormalized tables
    populate_cassandra_denormalized_tables(session, data_path)

def verify_cassandra_counts(session, keyspace='benchmark'):
    """Count rows in all Cassandra tables."""
    tables = ['teachers', 'subjects', 'classes', 'students', 
                'enrollments', 'grades', 'schedules', 
                'student_enrollments', 'student_grades']
    max_len = max(len(t) for t in tables)
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        counts = {}
        
        for table in tables:
            try:
                rows = session.execute(f"SELECT COUNT(*) FROM {table}")
                count = rows.one()[0]
                counts[table] = count
            except Exception as e:
                print(f"ERROR: {e}")
                counts[table] = 'Error'
        
        print("--- Cassandra Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("----------------------------------")
        
        return counts
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [ ]:
# Cassandra Operations Execution

# Schema initialization
initialize_cassandra_schema(cassandra_session)

# Table verification
required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
verify_cassandra_tables(cassandra_session, expected_tables=required_tables)

# Data loading
load_cassandra_data(cassandra_session, scale_100_dir)

# Row count verification
verify_cassandra_counts(cassandra_session)
CELL_END

In [ ]:
def postgres_operation(conn, query, fetch=False):
    """
    Executes a PostgreSQL operation and returns the result.
    """
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            if fetch:
                result = cur.fetchall()
                print(result)
        
        conn.commit()
    except Exception as e:
        print(f"ERROR: {e}")
        conn.rollback()
        return None

In [ ]:
def mariadb_operation(conn, query, fetch=False):
    """
    Executes a MariaDB operation and returns the result.
    """
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            if fetch:
                result = cur.fetchall()
                print(result)
        
        conn.commit()
    except Exception as e:
        print(f"ERROR: {e}")
        conn.rollback()
        return None

In [ ]:
def mongo_operation(client, db_name, collection_name, operation_type, query=None, data=None, options=None, fetch=False):
    """
    Executes a MongoDB operation and returns the result.
    
    Args:
        client: MongoDB client connection
        db_name: Database name to operate on
        collection_name: Collection name to operate on
        operation_type: Type of operation ('find', 'insert', 'update', 'delete')
        query: Query filter for find/update/delete operations (dict)
        data: Data for insert/update operations (dict or list of dicts)
        options: Additional options for operations (dict)
        fetch: Whether to fetch and print results (boolean)
        
    Returns:
        Operation result or None if error
    """
    try:
        # Get database and collection references
        db = client[db_name]
        collection = db[collection_name]
        result = None
        
        # Execute the requested operation
        if operation_type == 'find':
            # For find operations
            query = query or {}
            options = options or {}
            cursor = collection.find(query, **options)
            result = list(cursor)
            if fetch:
                print(result)
                
        elif operation_type == 'insert':
            # For insert operations
            if isinstance(data, list):
                result = collection.insert_many(data)
            else:
                result = collection.insert_one(data)
                
        elif operation_type == 'update':
            # For update operations
            options = options or {}
            if options.get('multi', False):
                result = collection.update_many(query, data, **options)
            else:
                result = collection.update_one(query, data, **options)
                
        elif operation_type == 'delete':
            # For delete operations
            options = options or {}
            if options.get('multi', False):
                result = collection.delete_many(query, **options)
            else:
                result = collection.delete_one(query, **options)
                
        else:
            print(f"ERROR: Unsupported operation type: {operation_type}")
            return None
            
        return result
        
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [ ]:
def cassandra_operation(session, query, params=None, fetch=False, keyspace='benchmark'):
    """
    Executes a Cassandra CQL operation and returns the result.
    """
    try:
        # Set the keyspace
        session.execute(f"USE {keyspace};")
        
        # Execute the query with or without parameters
        if params:
            # For prepared statements, we need to prepare first
            prepared = session.prepare(query)
            result = session.execute(prepared, params)
        else:
            result = session.execute(query)
        
        # Rest of your function remains the same...
        if fetch and query.strip().upper().startswith('SELECT'):
            rows = list(result)
            for row in rows:
                print(row)
            return rows
        
        return None
        
    except Exception as e:
        print(f"ERROR: Cassandra operation failed: {e}")
        return None

In [ ]:
# id,first_name,last_name,subject,hire_date,created_at
test_teachers = {
    10000001: {"first_name": "Anna",    "last_name": "Smith",   "subject": "Mathematics",    "hire_date": "2010-01-15", "created_at": "2025-05-05T12:00:00Z"},
    10000002: {"first_name": "James",   "last_name": "Lee",     "subject": "History",        "hire_date": "2012-08-20", "created_at": "2025-05-05T12:00:00Z"},
    10000003: {"first_name": "Maria",   "last_name": "Garcia",  "subject": "Biology",        "hire_date": "2015-03-10", "created_at": "2025-05-05T12:00:00Z"},
    10000004: {"first_name": "David",   "last_name": "Johnson", "subject": "Chemistry",      "hire_date": "2011-11-01", "created_at": "2025-05-05T12:00:00Z"},
    10000005: {"first_name": "Linda",   "last_name": "Brown",   "subject": "English",        "hire_date": "2013-04-22", "created_at": "2025-05-05T12:00:00Z"},
    10000006: {"first_name": "Robert",  "last_name": "Jones",   "subject": "Physics",        "hire_date": "2014-09-30", "created_at": "2025-05-05T12:00:00Z"},
    10000007: {"first_name": "Patricia","last_name": "Miller",  "subject": "Art",            "hire_date": "2016-06-17", "created_at": "2025-05-05T12:00:00Z"},
    10000008: {"first_name": "Michael", "last_name": "Davis",   "subject": "Geography",      "hire_date": "2009-02-05", "created_at": "2025-05-05T12:00:00Z"},
    10000009: {"first_name": "Barbara", "last_name": "Wilson",  "subject": "Music",          "hire_date": "2017-12-12", "created_at": "2025-05-05T12:00:00Z"},
    10000010: {"first_name": "William", "last_name": "Taylor",  "subject": "Computer Science","hire_date": "2008-07-29", "created_at": "2025-05-05T12:00:00Z"}
}

# id,name,description,created_at 
test_subjects = {
    10000011: {"name": "Mathematics",        "description": "Math fundamentals",          "created_at": "2025-05-05T12:00:00Z"},
    10000012: {"name": "History",            "description": "World history overview",    "created_at": "2025-05-05T12:00:00Z"},
    10000013: {"name": "Biology",            "description": "Life sciences",             "created_at": "2025-05-05T12:00:00Z"},
    10000014: {"name": "Chemistry",          "description": "Chemical reactions",        "created_at": "2025-05-05T12:00:00Z"},
    10000015: {"name": "English",            "description": "Literature and grammar",    "created_at": "2025-05-05T12:00:00Z"},
    10000016: {"name": "Physics",            "description": "Mechanics and waves",        "created_at": "2025-05-05T12:00:00Z"},
    10000017: {"name": "Art",                "description": "Art history and practice",  "created_at": "2025-05-05T12:00:00Z"},
    10000018: {"name": "Geography",          "description": "Physical and human geo",    "created_at": "2025-05-05T12:00:00Z"},
    10000019: {"name": "Music",              "description": "Theory and performance",     "created_at": "2025-05-05T12:00:00Z"},
    10000020: {"name": "Computer Science",   "description": "Programming concepts",      "created_at": "2025-05-05T12:00:00Z"}
}

# id,name,teacher_id,created_at
test_classes = {
    10000021: {"name": "Algebra I",      "teacher_id": 10000001, "created_at": "2025-05-05T12:00:00Z"},
    10000022: {"name": "World History",  "teacher_id": 10000002, "created_at": "2025-05-05T12:00:00Z"},
    10000023: {"name": "Biology 101",    "teacher_id": 10000003, "created_at": "2025-05-05T12:00:00Z"},
    10000024: {"name": "Organic Chemistry","teacher_id":10000004, "created_at": "2025-05-05T12:00:00Z"},
    10000025: {"name": "English Lit",    "teacher_id": 10000005, "created_at": "2025-05-05T12:00:00Z"},
    10000026: {"name": "Physics I",      "teacher_id": 10000006, "created_at": "2025-05-05T12:00:00Z"},
    10000027: {"name": "Drawing",        "teacher_id": 10000007, "created_at": "2025-05-05T12:00:00Z"},
    10000028: {"name": "World Geography","teacher_id": 10000008, "created_at": "2025-05-05T12:00:00Z"},
    10000029: {"name": "Choir",          "teacher_id": 10000009, "created_at": "2025-05-05T12:00:00Z"},
    10000030: {"name": "Intro to CS",    "teacher_id": 10000010, "created_at": "2025-05-05T12:00:00Z"}
}

# id,first_name,last_name,birth_date,created_at
test_students = {
    10000031: {"first_name": "John",   "last_name": "Doe",    "birth_date": "2005-06-15", "created_at": "2025-05-05T12:00:00Z"},
    10000032: {"first_name": "Alice",  "last_name": "Wang",   "birth_date": "2006-11-02", "created_at": "2025-05-05T12:00:00Z"},
    10000033: {"first_name": "Bob",    "last_name": "Nguyen", "birth_date": "2005-02-28", "created_at": "2025-05-05T12:00:00Z"},
    10000034: {"first_name": "Carol",  "last_name": "Kim",    "birth_date": "2006-01-11", "created_at": "2025-05-05T12:00:00Z"},
    10000035: {"first_name": "Eve",    "last_name": "Patel",  "birth_date": "2005-09-23", "created_at": "2025-05-05T12:00:00Z"},
    10000036: {"first_name": "Frank",  "last_name": "Lopez",  "birth_date": "2006-07-05", "created_at": "2025-05-05T12:00:00Z"},
    10000037: {"first_name": "Grace",  "last_name": "Chen",   "birth_date": "2005-12-19", "created_at": "2025-05-05T12:00:00Z"},
    10000038: {"first_name": "Hank",   "last_name": "Singh",  "birth_date": "2006-03-30", "created_at": "2025-05-05T12:00:00Z"},
    10000039: {"first_name": "Ivy",    "last_name": "Martinez","birth_date": "2005-10-08", "created_at": "2025-05-05T12:00:00Z"},
    10000040: {"first_name": "Jack",   "last_name": "Clark",  "birth_date": "2006-05-17", "created_at": "2025-05-05T12:00:00Z"}
}

# student_id,class_id,enrolled_at
test_enrollments = {
    (10000031, 10000021): {"enrolled_at": "2023-09-01T08:00:00Z"},
    (10000032, 10000022): {"enrolled_at": "2023-09-02T09:00:00Z"},
    (10000033, 10000023): {"enrolled_at": "2023-09-03T10:00:00Z"},
    (10000034, 10000024): {"enrolled_at": "2023-09-04T11:00:00Z"},
    (10000035, 10000025): {"enrolled_at": "2023-09-05T12:00:00Z"},
    (10000036, 10000026): {"enrolled_at": "2023-09-06T13:00:00Z"},
    (10000037, 10000027): {"enrolled_at": "2023-09-07T14:00:00Z"},
    (10000038, 10000028): {"enrolled_at": "2023-09-08T15:00:00Z"},
    (10000039, 10000029): {"enrolled_at": "2023-09-09T16:00:00Z"},
    (10000040, 10000030): {"enrolled_at": "2023-09-10T17:00:00Z"}
}

# id,class_id,subject_id,day_of_week,time_start,time_end
test_schedules = {
    10000051: {"class_id": 10000021, "subject_id": 10000011, "day_of_week": "Monday",    "time_start": "08:00", "time_end": "09:30"},
    10000052: {"class_id": 10000022, "subject_id": 10000012, "day_of_week": "Tuesday",   "time_start": "09:00", "time_end": "10:30"},
    10000053: {"class_id": 10000023, "subject_id": 10000013, "day_of_week": "Wednesday", "time_start": "10:00", "time_end": "11:30"},
    10000054: {"class_id": 10000024, "subject_id": 10000014, "day_of_week": "Thursday",  "time_start": "11:00", "time_end": "12:30"},
    10000055: {"class_id": 10000025, "subject_id": 10000015, "day_of_week": "Friday",    "time_start": "12:00", "time_end": "13:30"},
    10000056: {"class_id": 10000026, "subject_id": 10000016, "day_of_week": "Monday",    "time_start": "13:00", "time_end": "14:30"},
    10000057: {"class_id": 10000027, "subject_id": 10000017, "day_of_week": "Tuesday",   "time_start": "14:00", "time_end": "15:30"},
    10000058: {"class_id": 10000028, "subject_id": 10000018, "day_of_week": "Wednesday", "time_start": "15:00", "time_end": "16:30"},
    10000059: {"class_id": 10000029, "subject_id": 10000019, "day_of_week": "Thursday",  "time_start": "16:00", "time_end": "17:30"},
    10000060: {"class_id": 10000030, "subject_id": 10000020, "day_of_week": "Friday",    "time_start": "17:00", "time_end": "18:30"}
}

# id,student_id,subject_id,grade,created_at
test_grades = {
    10000041: {"student_id": 10000031, "subject_id": 10000011, "grade": 85, "created_at": "2024-05-10T12:00:00Z"},
    10000042: {"student_id": 10000032, "subject_id": 10000012, "grade": 92, "created_at": "2024-06-15T14:30:00Z"},
    10000043: {"student_id": 10000033, "subject_id": 10000013, "grade": 78, "created_at": "2024-07-20T16:45:00Z"},
    10000044: {"student_id": 10000034, "subject_id": 10000014, "grade": 88, "created_at": "2024-08-22T10:15:00Z"},
    10000045: {"student_id": 10000035, "subject_id": 10000015, "grade": 91, "created_at": "2024-09-05T09:00:00Z"},
    10000046: {"student_id": 10000036, "subject_id": 10000016, "grade": 79, "created_at": "2024-10-12T11:20:00Z"},
    10000047: {"student_id": 10000037, "subject_id": 10000017, "grade": 94, "created_at": "2024-11-30T13:50:00Z"},
    10000048: {"student_id": 10000038, "subject_id": 10000018, "grade": 82, "created_at": "2024-12-18T15:05:00Z"},
    10000049: {"student_id": 10000039, "subject_id": 10000019, "grade": 76, "created_at": "2025-01-25T08:40:00Z"},
    10000050: {"student_id": 10000040, "subject_id": 10000020, "grade": 89, "created_at": "2025-02-14T14:10:00Z"}
}

In [ ]:
import time
import psutil
import statistics
from typing import List, Tuple, Callable

class SimpleBenchmark:
    """Benchmark utility that prints results and returns pandas DataFrame for comparison"""
    
    def __init__(self, db_type: str, data_dir: str):
        self.data_dir = data_dir
        self.db_type = db_type
        self.process = psutil.Process()
        self.results = []  # Store results for each scenario
    
    def get_results_df(self):
        """
        Convert the results to a pandas DataFrame for easier analysis and comparison.
        
        Returns:
            pandas.DataFrame: DataFrame containing all benchmark results
        """
        import pandas as pd
        return pd.DataFrame(self.results)

    def run_scenarios(self, scenarios: List[Tuple[str, List[Tuple[str, Callable]]]], 
                    setup_method: Callable = None, 
                    cleanup_method: Callable = None):
        """
        Run multiple benchmark scenarios and collect metrics
        
        Args:
            scenarios: List of (scenario_name, [(operation_name, function)]) tuples
            setup_method: Optional function to run before each scenario (not measured)
            cleanup_method: Optional function to run after each scenario (not measured)
            
        Returns:
            pandas.DataFrame: Results organized for comparison
        """
        io_counters_start = psutil.disk_io_counters()
        
        # Run setup once if provided (not measured)
        if setup_method:
            setup_method()
            
        for scenario_name, operations in scenarios:
            # Initialize metrics collection
            start_time = time.time()
            cpu_samples = []
            memory_samples = []
            durations = []
            
            # Execute all operations in the scenario
            for op_name, func in operations:
                # Sample CPU and memory
                cpu_samples.append(self.process.cpu_percent())
                memory_samples.append(self.process.memory_info().rss)
                
                # Execute function and measure time
                op_start = time.time()
                func()
                op_duration = time.time() - op_start
                durations.append(op_duration)
            
            # Calculate metrics
            end_time = time.time()
            total_time = end_time - start_time
            
            # Resource usage
            avg_cpu = statistics.mean(cpu_samples) if cpu_samples else 0
            avg_memory = statistics.mean(memory_samples) / (1024 * 1024) if memory_samples else 0
            
            # Disk I/O 
            io_counters_end = psutil.disk_io_counters()
            read_mb = (io_counters_end.read_bytes - io_counters_start.read_bytes) / (1024 * 1024)
            write_mb = (io_counters_end.write_bytes - io_counters_start.write_bytes) / (1024 * 1024)
            
            # Performance metrics
            avg_op_time = statistics.mean(durations) if durations else 0
            throughput = len(operations) / total_time if total_time > 0 else 0
            
            # Save results for this scenario
            scenario_result = {
                'database': self.db_type,
                'data_dir': self.data_dir,
                'scenario': scenario_name,
                'total_time': total_time,
                'operations': len(operations),
                'avg_operation_time': avg_op_time,
                'throughput': throughput,
                'cpu_avg': avg_cpu,
                'memory_avg': avg_memory,
                'disk_read_mb': read_mb,
                'disk_write_mb': write_mb
            }
            self.results.append(scenario_result)
            
            # Print stats for this scenario
            print(f"--- {scenario_name} ({self.db_type}) ---")
            print(f"Total time: {total_time:.4f} seconds")
            print(f"Operations: {len(operations)}")
            print(f"Avg operation time: {avg_op_time:.4f} seconds")
            print(f"Throughput: {throughput:.2f} ops/sec")
            print(f"CPU avg: {avg_cpu:.2f}%")
            print(f"Memory avg: {avg_memory:.2f} MB")
            print(f"Disk read: {read_mb:.2f} MB")
            print(f"Disk write: {write_mb:.2f} MB")
            print()
            
            # Reset I/O counters for next scenario
            io_counters_start = io_counters_end
        
        # Run cleanup if provided (not measured)
        if cleanup_method:
            cleanup_method()
            


In [ ]:
INSERT_TEACHER="INSERT Teacher ?"
INSERT_SUBJECT="INSERT Subject ?"
INSERT_CLASS="INSERT Class ?"
INSERT_STUDENT="INSERT Student ?"
INSERT_ENROLLMENT="INSERT Enrollment ?"
INSERT_GRADE="INSERT Grade ?"
INSERT_SCHEDULE="INSERT Schedule ?"

SELECT_STUDENT = "SELECT student ?"
SELECT_CLASS = "SELECT class ?"
SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER = "SELECT all students that are taught by the teacher ?"
SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT = "SELECT all schedules for the student ?"
SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS = "SELECT all grades with the subjects descriptions for all students in the class ?"

UPDATE_STUDENT_NAME = "UPDATE Student ? Name"
UPDATE_ALL_GRADES_FOR_STUDENT = "UPDATE all Grades for Student ?"
UPDATE_CLASS_NAME = "UPDATE Class ? Name"
UPDATE_TEACHER_LAST_NAME = "UPDATE Teacher ? Last Name"
UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM = "UPDATE Subject ? Description for all subjects that students has grades from"

DELETE_STUDENT = "DELETE Student ?"
DELETE_CLASS = "DELETE Class ?"
DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER = "DELETE Subjects that are in the schedule of the teacher ?"
DELETE_TEACHER_WHO_TAUGHT_STUDENT = "DELETE Teacher who taught student ?"

In [ ]:
def postgres_benchmark(data_dir, random_numbers_list) -> pd.DataFrame:
    benchmark = SimpleBenchmark("postgres", data_dir)
    def setup_for_insert():
        with open('schemas/postgres_schema.sql', 'r') as f:
            sql_schema = f.read()

        initialize_postgres_schema(postgres_client, sql_schema)
        load_postgres_data(postgres_client, data_dir)

    insert_scenarios = [
        (
            INSERT_TEACHER,
            [
                (INSERT_TEACHER + str(k), lambda k=k, v=v: postgres_operation(postgres_client, 
                    f"INSERT INTO teachers (id, first_name, last_name, subject, hire_date, created_at) VALUES ({k}, '{v['first_name']}', '{v['last_name']}', '{v['subject']}', '{v['hire_date']}', '{v['created_at']}')")) 
                for k, v in test_teachers.items()
            ]
        ),
        (
            INSERT_SUBJECT,
            [
                (INSERT_SUBJECT + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO subjects (id, name, description, created_at) VALUES ({k}, '{v['name']}', '{v['description']}', '{v['created_at']}')"))
                for k, v in test_subjects.items()
            ]
        ),
        (
            INSERT_CLASS,
            [
                (INSERT_CLASS + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO classes (id, name, teacher_id, created_at) VALUES ({k}, '{v['name']}', {v['teacher_id']}, '{v['created_at']}')"))
                for k, v in test_classes.items()
            ]
        ), 
        (
            INSERT_STUDENT,
            [
                (INSERT_STUDENT + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO students (id, first_name, last_name, birth_date, created_at) VALUES ({k}, '{v['first_name']}', '{v['last_name']}', '{v['birth_date']}', '{v['created_at']}')"))
                for k, v in test_students.items()
            ]
        ),
        (
            INSERT_ENROLLMENT,
            [
                (INSERT_ENROLLMENT + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO enrollments (student_id, class_id, enrolled_at) VALUES ({k[0]}, {k[1]}, '{v['enrolled_at']}')"))
                for k, v in test_enrollments.items()
            ]
        ),
        (
            INSERT_GRADE,
            [
                (INSERT_GRADE + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO grades (id, student_id, subject_id, grade, created_at) VALUES ({k}, {v['student_id']}, {v['subject_id']}, {v['grade']}, '{v['created_at']}')"))
                for k, v in test_grades.items()
            ]
        ),
        (
            INSERT_SCHEDULE,
            [
                (INSERT_SCHEDULE + str(k), lambda k=k, v=v: postgres_operation(postgres_client,
                    f"INSERT INTO schedules (id, class_id, subject_id, day_of_week, time_start, time_end) VALUES ({k}, {v['class_id']}, {v['subject_id']}, '{v['day_of_week']}', '{v['time_start']}', '{v['time_end']}')"))
                for k, v in test_schedules.items()
            ]
        )
    ]

    select_scenarios = [
        (
            SELECT_STUDENT, [
                (SELECT_STUDENT + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM students WHERE id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_CLASS, [
                (SELECT_CLASS + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM classes WHERE id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER, [
                (SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM students s JOIN enrollments e ON s.id = e.student_id JOIN classes c ON e.class_id = c.id WHERE c.teacher_id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT, [
                (SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM schedules s JOIN classes c ON s.class_id = c.id JOIN enrollments e ON c.id = e.class_id WHERE e.student_id = {k}", fetch=True))
                for k in random_numbers_list
            ] 
        ),
        (
            SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS, [
                (SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"SELECT * FROM grades g JOIN subjects su ON g.subject_id = su.id JOIN students s ON g.student_id = s.id JOIN enrollments e ON s.id = e.student_id JOIN classes c ON e.class_id = c.id WHERE c.id = {k}", fetch=True))
                for k in random_numbers_list
            ] 
        ),
    ]

    update_scenarios = [
        (
            UPDATE_STUDENT_NAME, [
                (UPDATE_STUDENT_NAME + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE students SET last_name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_ALL_GRADES_FOR_STUDENT, [
                (UPDATE_ALL_GRADES_FOR_STUDENT + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE grades SET grade = 100 WHERE student_id = {k} AND subject_id = 10000011"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_CLASS_NAME, [
                (UPDATE_CLASS_NAME + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE classes SET name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_TEACHER_LAST_NAME, [
                (UPDATE_TEACHER_LAST_NAME + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE teachers SET last_name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ],
        ),
        (
            UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM, [
                (UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"UPDATE subjects SET description = 'UPDATED' WHERE id IN (SELECT subject_id FROM grades WHERE student_id = {k})"))
                for k in random_numbers_list
            ],
        )
    ]

    delete_scenarios = [
        (
            DELETE_STUDENT, [
                (DELETE_STUDENT + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM students WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_CLASS, [
                (DELETE_CLASS + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM classes WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER, [
                (DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM subjects WHERE id IN (SELECT subject_id FROM schedules WHERE class_id IN (SELECT id FROM classes WHERE teacher_id = {k}))"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_TEACHER_WHO_TAUGHT_STUDENT, [
                (DELETE_TEACHER_WHO_TAUGHT_STUDENT + str(k), lambda k=k: postgres_operation(postgres_client,
                    f"DELETE FROM teachers WHERE id IN (SELECT teacher_id FROM classes WHERE id IN (SELECT class_id FROM enrollments WHERE student_id = {k}))"))
                for k in random_numbers_list
            ]
        )
    ]

    benchmark.run_scenarios(insert_scenarios, setup_method=setup_for_insert)
    benchmark.run_scenarios(select_scenarios)
    benchmark.run_scenarios(update_scenarios)
    benchmark.run_scenarios(delete_scenarios)
    return benchmark.get_results_df()


In [ ]:
def mariadb_benchmark(data_dir, random_numbers_list) -> pd.DataFrame:
    benchmark = SimpleBenchmark("mariadb", data_dir)
    def setup_for_insert():
        with open('schemas/mariadb_schema.sql', 'r') as f:
            sql_schema = f.read()

        initialize_mariadb_schema(mariadb_client, sql_schema)
        load_mariadb_data(mariadb_client, data_dir)

    insert_scenarios = [
        (
            INSERT_TEACHER,
            [
                (INSERT_TEACHER + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client, 
                    f"INSERT INTO teachers (id, first_name, last_name, subject, hire_date, created_at) VALUES ({k}, '{v['first_name']}', '{v['last_name']}', '{v['subject']}', '{v['hire_date']}', '{v['created_at']}')")) 
                for k, v in test_teachers.items()
            ]
        ),
        (
            INSERT_SUBJECT,
            [
                (INSERT_SUBJECT + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO subjects (id, name, description, created_at) VALUES ({k}, '{v['name']}', '{v['description']}', '{v['created_at']}')"))
                for k, v in test_subjects.items()
            ]
        ),
        (
            INSERT_CLASS,
            [
                (INSERT_CLASS + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO classes (id, name, teacher_id, created_at) VALUES ({k}, '{v['name']}', {v['teacher_id']}, '{v['created_at']}')"))
                for k, v in test_classes.items()
            ]
        ), 
        (
            INSERT_STUDENT,
            [
                (INSERT_STUDENT + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO students (id, first_name, last_name, birth_date, created_at) VALUES ({k}, '{v['first_name']}', '{v['last_name']}', '{v['birth_date']}', '{v['created_at']}')"))
                for k, v in test_students.items()
            ]
        ),
        (
            INSERT_ENROLLMENT,
            [
                (INSERT_ENROLLMENT + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO enrollments (student_id, class_id, enrolled_at) VALUES ({k[0]}, {k[1]}, '{v['enrolled_at']}')"))
                for k, v in test_enrollments.items()
            ]
        ),
        (
            INSERT_GRADE,
            [
                (INSERT_GRADE + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO grades (id, student_id, subject_id, grade, created_at) VALUES ({k}, {v['student_id']}, {v['subject_id']}, {v['grade']}, '{v['created_at']}')"))
                for k, v in test_grades.items()
            ]
        ),
        (
            INSERT_SCHEDULE,
            [
                (INSERT_SCHEDULE + str(k), lambda k=k, v=v: mariadb_operation(mariadb_client,
                    f"INSERT INTO schedules (id, class_id, subject_id, day_of_week, time_start, time_end) VALUES ({k}, {v['class_id']}, {v['subject_id']}, '{v['day_of_week']}', '{v['time_start']}', '{v['time_end']}')"))
                for k, v in test_schedules.items()
            ]
        )
    ]

    select_scenarios = [
        (
            SELECT_STUDENT, [
                (SELECT_STUDENT + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"SELECT * FROM students WHERE id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_CLASS, [
                (SELECT_CLASS + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"SELECT * FROM classes WHERE id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER, [
                (SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"SELECT * FROM students s JOIN enrollments e ON s.id = e.student_id JOIN classes c ON e.class_id = c.id WHERE c.teacher_id = {k}", fetch=True))
                for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT, [
                (SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"SELECT * FROM schedules s JOIN classes c ON s.class_id = c.id JOIN enrollments e ON c.id = e.class_id WHERE e.student_id = {k}", fetch=True))
                for k in random_numbers_list
            ] 
        ),
        (
            SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS, [
                (SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"SELECT * FROM grades g JOIN subjects su ON g.subject_id = su.id JOIN students s ON g.student_id = s.id JOIN enrollments e ON s.id = e.student_id JOIN classes c ON e.class_id = c.id WHERE c.id = {k}", fetch=True))
                for k in random_numbers_list
            ] 
        ),
    ]

    update_scenarios = [
        (
            UPDATE_STUDENT_NAME, [
                (UPDATE_STUDENT_NAME + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"UPDATE students SET last_name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            UPDATE_ALL_GRADES_FOR_STUDENT, [
                (UPDATE_ALL_GRADES_FOR_STUDENT + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"UPDATE grades SET grade = 100 WHERE student_id = {k} AND subject_id = 10000011"))
                for k in random_numbers_list
            ]
        ),
        (
            UPDATE_CLASS_NAME, [
                (UPDATE_CLASS_NAME + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"UPDATE classes SET name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            UPDATE_TEACHER_LAST_NAME, [
                (UPDATE_TEACHER_LAST_NAME + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"UPDATE teachers SET last_name = 'UPDATED' WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM, [
                (UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"UPDATE subjects SET description = 'UPDATED' WHERE id IN (SELECT subject_id FROM grades WHERE student_id = {k})"))
                for k in random_numbers_list
            ]
        )
    ]

    delete_scenarios = [
        (
            DELETE_STUDENT, [
                (DELETE_STUDENT + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"DELETE FROM students WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_CLASS, [
                (DELETE_CLASS + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"DELETE FROM classes WHERE id = {k}"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER, [
                (DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"DELETE FROM subjects WHERE id IN (SELECT subject_id FROM schedules WHERE class_id IN (SELECT id FROM classes WHERE teacher_id = {k}))"))
                for k in random_numbers_list
            ]
        ),
        (
            DELETE_TEACHER_WHO_TAUGHT_STUDENT, [
                (DELETE_TEACHER_WHO_TAUGHT_STUDENT + str(k), lambda k=k: mariadb_operation(mariadb_client,
                    f"DELETE FROM teachers WHERE id IN (SELECT teacher_id FROM classes WHERE id IN (SELECT class_id FROM enrollments WHERE student_id = {k}))"))
                for k in random_numbers_list
            ]
        )
    ]

    benchmark.run_scenarios(insert_scenarios, setup_method=setup_for_insert)
    benchmark.run_scenarios(select_scenarios)
    benchmark.run_scenarios(update_scenarios)
    benchmark.run_scenarios(delete_scenarios)
    return benchmark.get_results_df()

In [ ]:
def mongo_benchmark(data_dir, random_numbers_list) -> pd.DataFrame:
    benchmark = SimpleBenchmark("mongodb", data_dir)
    
    def setup_for_insert():
        # Initialize MongoDB schema - create collections and indexes
        initialize_mongo_schema(mongo_client)
        # Load data from CSV files
        load_mongo_data(mongo_client, data_dir)
    
    insert_scenarios = [
        (
            INSERT_TEACHER,
            [
                (INSERT_TEACHER + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'teachers', 'insert',
                    data={
                        "_id": k,
                        "first_name": v['first_name'],
                        "last_name": v['last_name'],
                        "subject": v['subject'],
                        "hire_date": v['hire_date'],
                        "created_at": v['created_at']
                    }
                )) for k, v in test_teachers.items()
            ]
        ),
        (
            INSERT_SUBJECT,
            [
                (INSERT_SUBJECT + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'subjects', 'insert',
                    data={
                        "_id": k,
                        "name": v['name'],
                        "description": v['description'],
                        "created_at": v['created_at']
                    }
                )) for k, v in test_subjects.items()
            ]
        ),
        (
            INSERT_CLASS,
            [
                (INSERT_CLASS + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'insert',
                    data={
                        "_id": k,
                        "name": v['name'],
                        "teacher_id": v['teacher_id'],
                        "created_at": v['created_at']
                    }
                )) for k, v in test_classes.items()
            ]
        ),
        (
            INSERT_STUDENT,
            [
                (INSERT_STUDENT + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'insert',
                    data={
                        "_id": k,
                        "first_name": v['first_name'],
                        "last_name": v['last_name'],
                        "birth_date": v['birth_date'],
                        "created_at": v['created_at'],
                        "enrollments": [],
                        "grades": []
                    }
                )) for k, v in test_students.items()
            ]
        ),
        (
            INSERT_ENROLLMENT,
            [
                (INSERT_ENROLLMENT + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'update',
                    query={"_id": k[0]},
                    data={"$push": {"enrollments": {
                        "class_id": k[1],
                        "enrolled_at": v['enrolled_at']
                    }}}
                )) for k, v in test_enrollments.items()
            ]
        ),
        (
            INSERT_GRADE,
            [
                (INSERT_GRADE + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'update',
                    query={"_id": v['student_id']},
                    data={"$push": {"grades": {
                        "grade_id": k,
                        "subject_id": v['subject_id'],
                        "grade": v['grade'],
                        "created_at": v['created_at']
                    }}}
                )) for k, v in test_grades.items()
            ]
        ),
        (
            INSERT_SCHEDULE,
            [
                (INSERT_SCHEDULE + str(k), lambda k=k, v=v: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'update',
                    query={"_id": v['class_id']},
                    data={"$push": {"schedule": {
                        "schedule_id": k,
                        "subject_id": v['subject_id'],
                        "day_of_week": v['day_of_week'],
                        "time_start": v['time_start'],
                        "time_end": v['time_end']
                    }}}
                )) for k, v in test_schedules.items()
            ]
        )
    ]
    
    select_scenarios = [
        (
            SELECT_STUDENT, [
                (SELECT_STUDENT + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'find',
                    query={"_id": k},
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_CLASS, [
                (SELECT_CLASS + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'find',
                    query={"_id": k},
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER, [
                (SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'find',
                    query={"enrollments.class_id": {"$in": 
                            list(mongo_client['benchmark']['classes'].find({"teacher_id": k}, {"_id": 1}))
                    }},
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT, [
                (SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'find',
                    query={
                        "_id": {"$in": list(
                            map(lambda x: x["class_id"], 
                                mongo_client['benchmark']['students'].find_one({"_id": k}, {"enrollments": 1})["enrollments"]
                            )
                        )}
                    },
                    options={"projection": {"schedule": 1, "name": 1}},
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS, [
                (SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'find',
                    query={"enrollments.class_id": k},
                    options={
                        "projection": {
                            "first_name": 1, 
                            "last_name": 1, 
                            "grades": 1
                        }
                    },
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
    ]
    
    update_scenarios = [
        (
            UPDATE_STUDENT_NAME, [
                (UPDATE_STUDENT_NAME + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'update',
                    query={"_id": k},
                    data={"$set": {"last_name": "UPDATED"}}
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_ALL_GRADES_FOR_STUDENT, [
                (UPDATE_ALL_GRADES_FOR_STUDENT + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'update',
                    query={
                        "_id": k,
                        "grades.subject_id": 10000011
                    },
                    data={"$set": {"grades.$.grade": 100}}
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_CLASS_NAME, [
                (UPDATE_CLASS_NAME + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'update',
                    query={"_id": k},
                    data={"$set": {"name": "UPDATED"}}
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_TEACHER_LAST_NAME, [
                (UPDATE_TEACHER_LAST_NAME + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'teachers', 'update',
                    query={"_id": k},
                    data={"$set": {"last_name": "UPDATED"}}
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM, [
                (UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM + str(k), lambda k=k: 
                    # First get all subject IDs that the student has grades for
                    [mongo_operation(
                        mongo_client, 'benchmark', 'subjects', 'update',
                        query={"_id": subject_id},
                        data={"$set": {"description": "UPDATED"}}
                    ) for subject_id in list(map(
                        lambda x: x["subject_id"], 
                        mongo_client['benchmark']['students'].find_one({"_id": k}, {"grades.subject_id": 1})["grades"]
                    ))]
                ) for k in random_numbers_list
            ]
        ),
    ]
    
    delete_scenarios = [
        (
            DELETE_STUDENT, [
                (DELETE_STUDENT + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'students', 'delete',
                    query={"_id": k}
                )) for k in random_numbers_list
            ]
        ),
        (
            DELETE_CLASS, [
                (DELETE_CLASS + str(k), lambda k=k: mongo_operation(
                    mongo_client, 'benchmark', 'classes', 'delete',
                    query={"_id": k}
                )) for k in random_numbers_list
            ]
        ),
        (
            DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER, [
                (DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER + str(k), lambda k=k: 
                    # Find classes taught by teacher, then get all subject IDs from those classes' schedules
                    [mongo_operation(
                        mongo_client, 'benchmark', 'subjects', 'delete',
                        query={"_id": {"$in": list(set([
                            schedule["subject_id"] for class_doc in 
                            mongo_client['benchmark']['classes'].find({"teacher_id": k}) 
                            for schedule in class_doc.get("schedule", [])
                        ]))}}
                    )]
                ) for k in random_numbers_list
            ]
        ),
        (
            DELETE_TEACHER_WHO_TAUGHT_STUDENT, [
                (DELETE_TEACHER_WHO_TAUGHT_STUDENT + str(k), lambda k=k:
                    # Find student's enrolled class IDs, then find teachers of those classes
                    (lambda student_doc: 
                        [mongo_operation(
                            mongo_client, 'benchmark', 'teachers', 'delete',
                            query={"_id": {"$in": list(set([
                                class_doc["teacher_id"] for class_id in 
                                [enroll["class_id"] for enroll in student_doc.get("enrollments", [])]
                                for class_doc in mongo_client['benchmark']['classes'].find({"_id": class_id})
                            ]))}} if student_doc else {}
                        )]
                    )(mongo_client['benchmark']['students'].find_one({"_id": k}, {"enrollments": 1}))
                ) for k in random_numbers_list
            ]
        ),
    ]
    
    benchmark.run_scenarios(insert_scenarios, setup_method=setup_for_insert)
    benchmark.run_scenarios(select_scenarios)
    benchmark.run_scenarios(update_scenarios)
    benchmark.run_scenarios(delete_scenarios)
    
    return benchmark.get_results_df()

In [ ]:
def cassandra_benchmark(data_dir, random_numbers_list) -> pd.DataFrame:
    benchmark = SimpleBenchmark("cassandra", data_dir)
    
    def setup_for_insert():
        # Initialize Cassandra schema
        initialize_cassandra_schema(cassandra_session)
        # Load data from CSV files
        load_cassandra_data(cassandra_session, data_dir)
    
    # INSERT scenarios
    insert_scenarios = [
        (
            INSERT_TEACHER,
            [
                (INSERT_TEACHER + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session, 
                    "INSERT INTO teachers (id, first_name, last_name) VALUES (?, ?, ?)",
                    params=[k, v['first_name'], v['last_name']]
                )) for k, v in test_teachers.items()
            ]
        ),
        (
            INSERT_SUBJECT,
            [
                (INSERT_SUBJECT + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO subjects (id, name, description) VALUES (?, ?, ?)",
                    params=[k, v['name'], v['description']]
                )) for k, v in test_subjects.items()
            ]
        ),
        (
            INSERT_CLASS,
            [
                (INSERT_CLASS + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO classes (id, name, teacher_id) VALUES (?, ?, ?)",
                    params=[k, v['name'], v['teacher_id']]
                )) for k, v in test_classes.items()
            ]
        ),
        (
            INSERT_STUDENT,
            [
                (INSERT_STUDENT + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO students (id, first_name, last_name, birth_date) VALUES (?, ?, ?, ?)",
                    params=[k, v['first_name'], v['last_name'], v['birth_date']]
                )) for k, v in test_students.items()
            ]
        ),
        (
            INSERT_ENROLLMENT,
            [
                (INSERT_ENROLLMENT + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO enrollments (student_id, class_id, enrolled_at) VALUES (?, ?, ?)",
                    params=[k[0], k[1], v['enrolled_at']]
                )) for k, v in test_enrollments.items()
            ]
        ),
        (
            INSERT_GRADE,
            [
                (INSERT_GRADE + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO grades (id, student_id, subject_id, grade, created_at) VALUES (?, ?, ?, ?, ?)",
                    params=[k, v['student_id'], v['subject_id'], v['grade'], v['created_at']]
                )) for k, v in test_grades.items()
            ]
        ),
        (
            INSERT_SCHEDULE,
            [
                (INSERT_SCHEDULE + str(k), lambda k=k, v=v: cassandra_operation(
                    cassandra_session,
                    "INSERT INTO schedules (id, class_id, subject_id, day_of_week, time_start, time_end) VALUES (?, ?, ?, ?, ?, ?)",
                    # Convert day name to integer using day_map
                    params=[k, v['class_id'], v['subject_id'], 
                            {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}.get(v['day_of_week'], 0), 
                            v['time_start'], v['time_end']]
                )) for k, v in test_schedules.items()
            ]
        )
    ]
    
    # SELECT scenarios
    select_scenarios = [
        (
            SELECT_STUDENT, [
                (SELECT_STUDENT + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "SELECT * FROM students WHERE id = ?",
                    params=[k],
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_CLASS, [
                (SELECT_CLASS + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "SELECT * FROM classes WHERE id = ?",
                    params=[k],
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER, [
                (SELECT_ALL_STUDENTS_TAUGHT_BY_TEACHER + str(k), lambda k=k: cassandra_operation(
                    # Using denormalized student_enrollments table to get all students for a teacher
                    cassandra_session,
                    "SELECT s.* FROM students s JOIN student_enrollments se ON s.id = se.student_id WHERE se.teacher_id = ? ALLOW FILTERING",
                    params=[k],
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT, [
                (SELECT_ALL_SCHEDULES_FOR_A_SPECIFIC_STUDENT + str(k), lambda k=k: cassandra_operation(
                    # Using denormalized student_enrollments and schedules tables
                    cassandra_session,
                    """
                    SELECT sc.* FROM schedules sc 
                    JOIN student_enrollments se ON sc.class_id = se.class_id 
                    WHERE se.student_id = ? ALLOW FILTERING
                    """,
                    params=[k],
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
        (
            SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS, [
                (SELECT_ALL_GRADES_WITH_SUBJECT_DESCRIPTIONS_FOR_ALL_STUDENTS_IN_THE_CLASS + str(k), lambda k=k: cassandra_operation(
                    # Using denormalized student_grades table
                    cassandra_session,
                    """
                    SELECT sg.* FROM student_grades sg 
                    JOIN student_enrollments se ON sg.student_id = se.student_id 
                    WHERE se.class_id = ? ALLOW FILTERING
                    """,
                    params=[k],
                    fetch=True
                )) for k in random_numbers_list
            ]
        ),
    ]
    
    # UPDATE scenarios
    update_scenarios = [
        (
            UPDATE_STUDENT_NAME, [
                (UPDATE_STUDENT_NAME + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "UPDATE students SET last_name = 'UPDATED' WHERE id = ?",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_ALL_GRADES_FOR_STUDENT, [
                (UPDATE_ALL_GRADES_FOR_STUDENT + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "UPDATE grades SET grade = 100 WHERE student_id = ? AND subject_id = 10000011",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_CLASS_NAME, [
                (UPDATE_CLASS_NAME + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "UPDATE classes SET name = 'UPDATED' WHERE id = ?",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_TEACHER_LAST_NAME, [
                (UPDATE_TEACHER_LAST_NAME + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "UPDATE teachers SET last_name = 'UPDATED' WHERE id = ?",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM, [
                (UPDATE_SUBJECT_DESCRIPTION_FOR_ALL_SUBJECTS_THAT_STUDENTS_HAS_GRADES_FROM + str(k), lambda k=k: 
                    # First get all subject IDs that the student has grades for
                    cassandra_operation(
                        cassandra_session,
                        "UPDATE subjects SET description = 'UPDATED' WHERE id IN (SELECT subject_id FROM student_grades WHERE student_id = ? ALLOW FILTERING)",
                        params=[k]
                    )
                ) for k in random_numbers_list
            ]
        ),
    ]
    
    # DELETE scenarios
    delete_scenarios = [
        (
            DELETE_STUDENT, [
                (DELETE_STUDENT + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "DELETE FROM students WHERE id = ?",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            DELETE_CLASS, [
                (DELETE_CLASS + str(k), lambda k=k: cassandra_operation(
                    cassandra_session,
                    "DELETE FROM classes WHERE id = ?",
                    params=[k]
                )) for k in random_numbers_list
            ]
        ),
        (
            DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER, [
                (DELETE_SUBJECTS_THAT_ARE_IN_THE_SCHEDULE_OF_THE_TEACHER + str(k), lambda k=k: 
                    # Need to get subject IDs from schedules where class has teacher_id = k
                    cassandra_operation(
                        cassandra_session,
                        """
                        DELETE FROM subjects WHERE id IN (
                            SELECT subject_id FROM schedules 
                            WHERE class_id IN (
                                SELECT id FROM classes WHERE teacher_id = ? ALLOW FILTERING
                            ) ALLOW FILTERING
                        )
                        """,
                        params=[k]
                    )
                ) for k in random_numbers_list
            ]
        ),
        (
            DELETE_TEACHER_WHO_TAUGHT_STUDENT, [
                (DELETE_TEACHER_WHO_TAUGHT_STUDENT + str(k), lambda k=k:
                    # Find teacher IDs from student_enrollments
                    cassandra_operation(
                        cassandra_session,
                        "DELETE FROM teachers WHERE id IN (SELECT teacher_id FROM student_enrollments WHERE student_id = ? ALLOW FILTERING)",
                        params=[k]
                    )
                ) for k in random_numbers_list
            ]
        ),
    ]
    
    # Run all benchmark scenarios
    benchmark.run_scenarios(insert_scenarios, setup_method=setup_for_insert)
    benchmark.run_scenarios(select_scenarios)
    benchmark.run_scenarios(update_scenarios)
    benchmark.run_scenarios(delete_scenarios)
    
    return benchmark.get_results_df()

In [ ]:
from random import sample

rand_10_between_1_100 = sample(range(1, 101), 10)
postgres_results_df = postgres_benchmark(scale_100_dir, rand_10_between_1_100)
mariadb_results_df = mariadb_benchmark(scale_100_dir, rand_10_between_1_100)
mongo_results_df = mongo_benchmark(scale_100_dir, rand_10_between_1_100)
cassandra_results_df = cassandra_benchmark(scale_100_dir, rand_10_between_1_100)

In [ ]:
merged_df = pd.concat([postgres_results_df, mariadb_results_df, mongo_results_df, cassandra_results_df], ignore_index=True)
merged_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def plot_database_performance(
    df, 
    keyword=None,
    metric='total_time', 
    data_dir=None, 
    figsize=(14, 8), 
    title=None,
    sort_by=None,
    log_scale=False
):
    """
    Create a performance comparison plot for different databases.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The merged DataFrame containing all benchmark results
    keyword : str, optional
        Filter scenarios containing this keyword
    metric : str, default='total_time'
        The metric to plot. Must be one of:
        'total_time', 'operations', 'avg_operation_time', 'throughput',
        'cpu_avg', 'memory_avg', 'disk_read_mb', 'disk_write_mb'
    data_dir : str, optional
        Filter by specific data directory
    figsize : tuple, default=(14, 8)
        Figure size as (width, height)
    title : str, optional
        Custom title for the plot
    sort_by : str, optional
        Sort scenarios by: 'name', 'value', or None for default ordering
    log_scale : bool, default=False
        Use logarithmic scale for the metric axis
    
    Returns:
    --------
    fig, ax: matplotlib figure and axis objects
    """
    # Validate metric
    valid_metrics = ['total_time', 'operations', 'avg_operation_time', 'throughput',
                    'cpu_avg', 'memory_avg', 'disk_read_mb', 'disk_write_mb']
    
    if metric not in valid_metrics:
        raise ValueError(f"Invalid metric: {metric}. Must be one of {valid_metrics}")
    
    # Clone dataframe to avoid modifying the original
    plot_df = df.copy()
    
    # Apply filters
    if data_dir:
        plot_df = plot_df[plot_df['data_dir'] == data_dir]
    
    if keyword:
        plot_df = plot_df[plot_df['scenario'].str.contains(keyword, case=False)]
    
    if plot_df.empty:
        print("No data matches the specified filters.")
        return None, None
    
    # Get unique databases and scenarios
    databases = plot_df['database'].unique()
    scenarios = plot_df['scenario'].unique()
    
    # Sort scenarios if requested
    if sort_by == 'name':
        scenarios = sorted(scenarios)
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize)
    
    # Set Seaborn style
    sns.set_style("whitegrid")
    
    # Define color palette
    palette = sns.color_palette("husl", len(scenarios))
    
    # Calculate bar width based on number of scenarios
    bar_width = 0.8 / len(scenarios)
    
    # Create bars for each scenario
    for i, scenario in enumerate(scenarios):
        # Get data for this scenario
        scenario_data = plot_df[plot_df['scenario'] == scenario]
        
        # Calculate x positions for this scenario
        x = np.arange(len(databases))
        offset = (i - len(scenarios)/2 + 0.5) * bar_width
        
        # Get values for each database for this metric and scenario
        values = []
        for db in databases:
            val = scenario_data[scenario_data['database'] == db][metric].values
            values.append(val[0] if len(val) > 0 else np.nan)
        
        # Plot bars
        ax.bar(x + offset, values, width=bar_width, label=scenario, color=palette[i])
    
    # Set logarithmic scale if requested
    if log_scale and all(v > 0 for v in plot_df[metric].values):
        ax.set_yscale('log')
    
    # Customize plot
    if not title:
        title = f"{metric.replace('_', ' ').title()} by Database and Scenario"
        if keyword:
            title += f" (filtered: '{keyword}')"
    
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Database', fontsize=14)
    ax.set_ylabel(metric.replace('_', ' ').title(), fontsize=14)
    ax.set_xticks(np.arange(len(databases)))
    ax.set_xticklabels(databases, rotation=45, ha='right')
    
    # Add legend with smaller font and outside the plot area
    ax.legend(title="Scenarios", bbox_to_anchor=(0.5, -0.15), loc='upper center', 
          fontsize=10, ncol=min(len(scenarios), 3))
    
    plt.tight_layout()
    return fig, ax

In [ ]:
fig, ax = plot_database_performance(
    merged_df,
    keyword="INSERT",
    metric="total_time",
    log_scale=False
)
plt.show()

In [ ]:
fig, ax = plot_database_performance(
    merged_df,
    keyword="SELECT",
    metric="total_time",
    log_scale=False
)
plt.show()

In [ ]:
fig, ax = plot_database_performance(
    merged_df,
    keyword="UPDATE",
    metric="total_time",
    log_scale=False
)
plt.show()

In [ ]:
fig, ax = plot_database_performance(
    merged_df,
    keyword="DELETE",
    metric="total_time",
    log_scale=False
)
plt.show()